In [1]:
import pandas as pd
import altair as alt
import numpy as np
from scipy import stats
from pathlib import Path

In [2]:
rna_count_folder = '../Data/RNAscoring_dev_data/RNA_counts/' #path to RNA counts folder
dna_count_folder = '../Data/RNAscoring_dev_data/DNA_counts/' #paths to DNA counts folder
sge_score = '../Data/20250122_BARD1_SGEscores_wAAsub.xlsx' #path to SGe score file

In [183]:
def read_counts(rna_counts, dna_counts): #reads RNA/DNA count files and merges them
               
    rna_path = Path(rna_counts) #Creates path object to RNA counts
    dna_path = Path(dna_counts) #Creates path object to DNA counts
    
    rna_outputs = sorted(list(rna_path.glob('*.tsv'))) #Gets list of RNA count files. Sort function added to order replicates
    dna_outputs = sorted(list(dna_path.glob('*.tsv'))) #Gets list of DNA count files. Sort function added to order replicates
    
    all_outputs = [] #list to hold tuples of DNA/RNA count files paired by replicate

    i = 0
    while i < len(rna_outputs): #Iterates through RNA count files and constructs the paired DNA/RNA tuples
        paired_output = (rna_outputs[i], dna_outputs[i]) #Paired RNA/DNA files
        all_outputs.append(paired_output) #Appends ot list

        i += 1

    all_rep_counts = {} #Dictionary to hold merged RNA/DNA count dataframe
    
    for elem in all_outputs: #Iterates through list of paired RNA/DNA count files
        rna, dna = elem #Breaks up tuple into constituent RNA/DNA count file paths
        rna_counts = pd.read_csv(rna, sep = '\t') #Reads RNA count file
        sampleid = rna_counts['sampleid'][0] #Gets sample ID from first row

        #Booleans to get replicate number from sample ID
        if 'R3R6' in sampleid or 'R7R8R9' in sampleid:
            rep = 'rep_3'
        elif 'R2R5'in sampleid or 'R4R5R6' in sampleid:
            rep = 'rep_2'
        elif 'R1R4' in sampleid or 'R1R2R3' in sampleid:
            rep = 'rep_1'

        #Creates a column that has target and replicate number
        target_rep_name = rna_counts['target'] + rep 
        rna_counts['target_rep'] = target_rep_name 
        rna_counts = rna_counts.rename(columns = {'count': 'RNAcount'}) #Renames count column in RNA file
        rna_counts['pos'] = rna_counts['pos'].astype(str) #Sets 'pos' column to string data type for merging
        rna_counts['pos_id'] = rna_counts['pos'] + ':' + rna_counts['allele'] #Creates position ID that is genomic coordinate : allele for merging

        
        dna_counts = pd.read_csv(dna, sep = '\t') #Reads DNA file
        dna_counts = dna_counts.rename(columns = {'count': 'DNAcount'}) #Renames count column in DNA file
        dna_counts['pos'] = dna_counts['pos'].astype(str) #Sets 'pos' column to string data type for merging
        dna_counts['pos_id'] = dna_counts['pos'] + ':' + dna_counts['allele'] #Creates position ID column for merging
         
        merged = pd.merge(rna_counts, dna_counts, how = 'inner', on = 'pos_id') #Merges RNA and DNA dataframes on intersection of pos_id column 
        merged = merged.drop(columns = ['sampleid_x', 'sampleid_y', 'target_x','chrom_x', 'chrom_y', 'pos_x', 'pos_y', 'allele_x', 'allele_y']) #drops duplicate columns 
        merged = merged.rename(columns  = {'target_y' : 'target'}) #Renames target column to keep exon SGE target name
        merged = merged.loc[:, ['target_rep','target', 'pos_id', 'RNAcount', 'DNAcount']] #Columns reordered
        
        all_rep_counts[target_rep_name[0]] = merged #merged dataframe added to dictionary


    return all_rep_counts

In [184]:
def rna_v_dna_plot(all_counts):
    rep_regions = list(all_counts.keys())

    df_list = []
    for elem in rep_regions:
        df_list.append(all_counts[elem])


    all_df = pd.concat(df_list)

    all_df = all_df[~((all_df['RNAcount'] < 50) | (all_df['DNAcount'] < 50))]

    scatter = alt.Chart(all_df).mark_circle().encode(
        x = alt.X('RNAcount', title = 'RNA Count'),
        y = alt.Y('DNAcount', title = 'DNA Count'),
        tooltip = [alt.Tooltip('RNAcount', title = 'RNA Count: '),
                   alt.Tooltip('DNAcount', title = 'DNA Count: '),
                   alt.Tooltip('pos_id', title = 'Var: ')
                  ]
    )

    trend_line = alt.Chart(all_df).transform_regression(
        'RNAcount', 'DNAcount',
        groupby=['target_rep']
    ).mark_line(color='blue').encode(
        x='RNAcount',
        y='DNAcount'
    )


    combined = (scatter + trend_line).facet(
        facet=alt.Facet('target_rep', title = 'DNA vs. RNA Counts')  # Set maximum of 4 columns per row
        ).resolve_scale(
            x = 'independent',
            y = 'independent'
        ).properties(
            columns = 6
        ).interactive()

    combined.display()

    all_targets = list(set(all_df['target'].tolist()))

In [133]:
def get_rna_scores(rep_dict): #Handles RNA scoring of variants

    target_reps = list(rep_dict.keys()) #Gets list of all targets and replicates present in dictionary
    rna_scored = {} #Diciotnary to hold RNA-scored dataframes
    targets = [] #list of SGE targets present
    
    #Does RNA scoring
    for elem in target_reps:
        df = rep_dict[elem] #Gets dataframe
        df = df[~((df['RNAcount'] == 0) | (df['DNAcount'] == 0))].copy() #Drops rows with RNA/DNA count of 0
        df = df[~((df['RNAcount'] < 100) | (df['DNAcount'] < 100))].copy()

        df = df.reset_index(drop = True) #Resets index
         
        DNAcount = df['DNAcount'].sum() #Gets total DNA counts
        RNAcount = df['RNAcount'].sum() #Gets total RNA counts

        df['DNAfreq'] = df['DNAcount'] / DNAcount #Calculates DNA frequency
        df['RNAfreq'] = df['RNAcount'] / RNAcount #Calculates RNA frequency

        slope, intercept, r_value, p_value, std_err = stats.linregress(df['RNAfreq'], df['DNAfreq'])
        
        df['RNA/DNA'] = df['RNAcount'] / df['DNAcount']
        df['RNA/DNA_freq'] = df['RNAfreq'] / df['DNAfreq']
        df['RNA/DNA_freq/corr'] = df['RNA/DNA_freq'] / slope
        df['RNAscore'] = np.log2(df['RNAfreq']/ df['DNAfreq']) #Calculates RNA score using log2 ratio

        rna_scored[df['target_rep'][0]] = df #RNA-scored dataframe is added to rna_scored dataframe

        if df['target'][0] not in targets: #Appends SGE target names
            targets.append(df['target'][0])
        
    
    return rna_scored, targets
        

In [167]:
def get_rna_scores_beta(all_counts):
    all_counts = list(all_counts.values())
    rna_scored = {}
    
    df = pd.concat(all_counts)

    
    def calculate_residuals(data):

        data = data[ ~ ((data['RNAcount'] == 0) | (data['DNAcount'] == 0))].copy()
        data = data[~((data['RNAcount'] < 100) | (data['DNAcount'] < 100))].copy()
        
        DNAcount = data['DNAcount'].sum()
        RNAcount = data['RNAcount'].sum()

        data['DNAfreq'] = data['DNAcount'] / DNAcount
        data['RNAfreq'] = data['RNAcount'] / RNAcount
        
        slope, intercept, r_value, p_value, std_err = stats.linregress(data['RNAfreq'], data['DNAfreq']) #Performs linear regression

        data['y_pred'] = slope * data['RNAfreq'] + intercept #Calculates predicted y-value (DNAfreq) from provided x-value (RNAfreq)
        data['residuals'] = -(data['DNAfreq'] - data['y_pred']) #Calculates vertical distance from trend line for each point

        perp_std = data['residuals'].std() #Standard deviation of all residuals calculated (previously used in normalizing scores)


        residual_median = data['residuals'].median() #Tabulates median residual
        residual_iqr = stats.iqr(data['residuals']) #Tabulates interquartile range for residuals
        
        # Avoiding division by zero if all points are exactly on the line
        if perp_std > 0:
            data['std_per_distance'] = (data['residuals'] - residual_median) / residual_iqr #Scores standarized relative to mean and interquartile range
        else:
            data['std_per_distance'] = np.nan

        return data

    df = df.groupby('target_rep').apply(calculate_residuals) #Calculates residuals
    df = df.reset_index(drop = True) #Ungroups
    #df = df.drop(columns = ['y_pred', 'residuals', 'per_distance', 'abs_residuals']) #Drops unused columns
    df = df.rename(columns = {'std_per_distance': 'RNAscore'})

    grouped = df.groupby('target_rep')
    for rep, data in grouped:
        rna_scored[rep] = data

    
    return rna_scored

In [158]:
def qc_plots(dict):
    
    all_rna = list(dict.values()) #Gets all dataframes
    df = pd.concat(all_rna) #Complete datafrmae

    
    grouped = df.groupby('target')
    for target, data in grouped:
        charts = []

        #Gets replicate dataframes
        rep1 = data.loc[data['target_rep'].str.contains('rep_1')].reset_index(drop = True)
        rep2 = data.loc[data['target_rep'].str.contains('rep_2')].reset_index(drop = True)
        rep3 = data.loc[data['target_rep'].str.contains('rep_3')].reset_index(drop = True)
        


        # Create dataframes for QC correlation comparison
        merged_1_2 = pd.merge(rep1, rep2, on='pos_id', how='inner', suffixes=('_rep1', '_rep2'))
        merged_1_3 = pd.merge(rep1, rep3, on='pos_id', how='inner', suffixes=('_rep1', '_rep3'))
        merged_2_3 = pd.merge(rep2, rep3, on='pos_id', how='inner', suffixes=('_rep2', '_rep3'))

            
        # Calculate correlations for each pair
        pairs = [
            ('rep1', 'rep2', merged_1_2, 'RNAscore_rep1', 'RNAscore_rep2'),
            ('rep1', 'rep3', merged_1_3, 'RNAscore_rep1', 'RNAscore_rep3'),
            ('rep2', 'rep3', merged_2_3, 'RNAscore_rep2', 'RNAscore_rep3')
            ]

        for x_rep, y_rep, merged_df, x_col, y_col in pairs:
            
            # Calculate correlation using aligned columns
            corr_pearson = np.corrcoef(merged_df[x_col], merged_df[y_col])[0,1] #Pearson correlation
            corr = stats.spearmanr(merged_df[x_col], merged_df[y_col]).correlation #Spearman correlation
            
            # Calculate min/max for diagonal line (y = x)
            min_val = min(merged_df[x_col].min(), merged_df[y_col].min())
            max_val = max(merged_df[x_col].max(), merged_df[y_col].max())
            
            line_df = pd.DataFrame({
                'x': [min_val, max_val],
                'y': [min_val, max_val]
            })
            
  
            
            # Create scatter plot
            scatter = alt.Chart(merged_df).mark_circle(opacity=0.7).encode(
                x=alt.X(x_col, title=f'{x_col} RNAscore'),
                y=alt.Y(y_col, title=f'{y_col} RNAscore'),
                tooltip = [alt.Tooltip('pos_id', title = 'Variant: '),
                           alt.Tooltip(x_col),
                           alt.Tooltip(y_col)
                          ]
            )
            
            # Create reference line
            line = alt.Chart(line_df).mark_line(
                color='red', 
                strokeDash=[4, 4]
            ).encode(
                x='x:Q',
                y='y:Q'
            )
            
            # Combine and add title with correlation
            combined = (scatter + line).properties(
                width=250,
                height=250,
                title= target + ' ' + f'{x_rep} vs {y_rep} (r = {corr_pearson:.3f})' 
            ).interactive()
            
            charts.append(combined)
        
        # Concatenate all charts horizontally
        final_chart = alt.hconcat(*charts).properties(
            title=''
        )
        
        # Display the chart
        final_chart.display()



In [106]:
def oldvsnew_scores(old_scores, new_scores):
    merged = pd.merge(old_scores, new_scores, how = 'left', on = 'pos_id')

    scatter = alt.Chart(merged).mark_circle().encode(
        x = 'RNAscore_x',
        y = 'RNAscore_y'
    )

    scatter.display()
    print(merged)

In [107]:
def collapse_scores(rna_scored, targets): #Collapses RNA scores between replicates to a median

    collapsed_scores = {} #Empty dictionary to hold dataframes with collapsed scores

    for elem in targets: #Iterates through provided SGE targets
        target_scores = [v for k, v in rna_scored.items() if elem in k] #Gets dicitonary elements in same SGE target

        concat_scores = pd.concat(target_scores) #Concatenates all scores
        concat_scores = concat_scores.drop(columns = ['target_rep','RNAcount', 'DNAcount', 'DNAfreq', 'RNAfreq']) #Drops columns no longer used

        #Generates summary dataframe that has a median RNA score for each variant
        summary_df = concat_scores.groupby('pos_id').agg({
            'target': 'first',
            'RNAscore': 'median'
        }).reset_index()

        #Tidys up collapsed RNA score df
        summmary_df = summary_df.loc[:, ['target', 'pos_id', 'RNAscore']]
        summary_df = summary_df.rename(columns = {'RNAscore': 'RNAscore_med'})
        
        collapsed_scores[elem] = summary_df #Appends collapsed RNA score df to dictionary

    return collapsed_scores

In [108]:
def merge_dna(dict, gdna_scores): #This function merges the DNA SGE scores the median RNA scores for each variant
    dna_scores = pd.read_excel(gdna_scores) #Reads DNA SGE Scores

    dna_scores = dna_scores.drop(columns = ['chrom', 'pos', 'allele', 'R1_score', 'R2_score', 'R3_score', 'target']) #Drops these columns
    all_rna = list(dict.values()) #Gets all dataframes stored in the RNAscore dictionary

    df = pd.concat(all_rna) #Concatenates all RNAscore dataframes

    merged = pd.merge(df, dna_scores, how = 'inner', on = 'pos_id') #Merges based on position ID


    multi_target_vars = merged.groupby('pos_id').agg({'RNAscore_med': 'median'}).reset_index()

    merged = pd.merge(multi_target_vars, merged, how = 'left', on = 'pos_id')
    merged = merged.rename(columns = {'RNAscore_med_x': 'RNAscore', 'RNAscore_med_y': 'target_RNAscore'})
    final_df = merged.loc[:, ['target', 'pos_id', 'Consequence', 'AAsub', 'snv_score', 'target_RNAscore', 'RNAscore']]

    
    return final_df

In [109]:
def visualize_scores(df):

    scatter_all = alt.Chart(df).mark_circle().encode(
        x = alt.X('snv_score', axis = alt.Axis(title = 'DNA Score')),
        y = alt.Y('RNAscore', axis = alt.Axis(title = 'RNA Score')),
        color = 'Consequence',
        tooltip = [alt.Tooltip('target', title = 'SGE Target: '),
                    alt.Tooltip('AAsub', title = 'AA Substitution: '),
                   alt.Tooltip('snv_score', title = 'DNA Score: '),
                   alt.Tooltip('RNAscore', title = 'RNA Score: ')
                  ]
    ).properties(
        height = 400, 
        width = 600,
        title = alt.TitleParams('RNA Score vs. DNA Score')
    ).interactive()

    scatter_all.display()
    
    histogram = alt.Chart(df).mark_bar().encode(
        x = alt.X('RNAscore:Q', title = 'RNA Score', bin = alt.Bin(maxbins = 30)),
        y =  alt.Y('count()', title = 'Number of Variants'),
        color = 'Consequence'
    ).properties(
        title = 'Distribution of RNA Scores',
        width = 600, 
        height = 400
    )

    histogram.display()
    
    scatter_faceted = scatter_all.facet(
        facet = alt.Facet('target', title = 'RNA vs. DNA Scores by Region')
    ).resolve_scale(
        x = 'independent', 
        y = 'independent'
    ).properties(
        columns = 3
    )

    scatter_faceted.display()

In [179]:
def main():
    all_counts = read_counts(rna_count_folder, dna_count_folder)

    rna_v_dna_plot(all_counts)
    
    rna_scored, sge_targets = get_rna_scores(all_counts)
    rna_scored_beta = get_rna_scores_beta(all_counts)

    qc_plots(rna_scored)    
    
    med_rna_scores = collapse_scores(rna_scored, sge_targets)
    med_rna_scores_new = collapse_scores(rna_scored_beta, sge_targets)

    dna_rna_df = merge_dna(med_rna_scores, sge_score)
    dna_rna_df_new = merge_dna(med_rna_scores_new, sge_score)

    #oldvsnew_scores(dna_rna_df, dna_rna_df_new)
    visualize_scores(dna_rna_df)


In [182]:
main()

[(PosixPath('../Data/RNAscoring_dev_data/RNA_counts/BARD1_X11A_R1R4.rnacounts.tsv'), PosixPath('../Data/RNAscoring_dev_data/DNA_counts/BARD1_X11A_R1R4_D05.snvs.tsv')), (PosixPath('../Data/RNAscoring_dev_data/RNA_counts/BARD1_X11A_R2R5.rnacounts.tsv'), PosixPath('../Data/RNAscoring_dev_data/DNA_counts/BARD1_X11A_R2R5_D05.snvs.tsv')), (PosixPath('../Data/RNAscoring_dev_data/RNA_counts/BARD1_X11A_R3R6.rnacounts.tsv'), PosixPath('../Data/RNAscoring_dev_data/DNA_counts/BARD1_X11A_R3R6_D05.snvs.tsv')), (PosixPath('../Data/RNAscoring_dev_data/RNA_counts/BARD1_X11B_R1R2R3.rnacounts.tsv'), PosixPath('../Data/RNAscoring_dev_data/DNA_counts/BARD1_X11B_R1R2R3_D05.snvs.tsv')), (PosixPath('../Data/RNAscoring_dev_data/RNA_counts/BARD1_X11B_R4R5R6.rnacounts.tsv'), PosixPath('../Data/RNAscoring_dev_data/DNA_counts/BARD1_X11B_R2R5_D05.snvs.tsv')), (PosixPath('../Data/RNAscoring_dev_data/RNA_counts/BARD1_X11B_R7R8R9.rnacounts.tsv'), PosixPath('../Data/RNAscoring_dev_data/DNA_counts/BARD1_X11B_R7R8R9_D05.

alt.FacetChart(...)

/tmp/12721204.1.shendure-login.q/ipykernel_2532035/1113724374.py:38: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('target_rep').apply(calculate_residuals) #Calculates residuals


alt.HConcatChart(...)

alt.HConcatChart(...)

alt.HConcatChart(...)

alt.HConcatChart(...)

alt.Chart(...)

alt.Chart(...)

alt.FacetChart(...)